In [1]:
# IR HW4 PLSA and Query likelihood
path = './ntust-ir-2020_hw4_v2/'

In [2]:
import numpy as np
from tqdm import tqdm
import random
import math
from numba import jit
from collections import Counter
import time

In [3]:
doc = []
doc_list = []
doc_len_list = []
query = []
query_list = []
terms_set =  set()
TF = Counter()
w_d = []
all_d_len = 0

In [4]:
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        query.append(temp_query)
        for w in temp_query:
            terms_set.add(w)
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        all_d_len += len(temp_doc)
        doc.append(temp_doc)
        doc_list.append(l)
        doc_len_list.append(len(temp_doc))
        TF.update(temp_doc)
        f_temp.close()

100%|██████████| 14955/14955 [01:13<00:00, 202.41it/s]


In [5]:
# terms_set
kill_list = []
for k,v in TF.items():
    if(not(k in terms_set) and v < 450):
        kill_list.append(k)
    elif(not(k in terms_set)):
        terms_set.add(k)
for k in kill_list:
    del TF[k]

In [6]:
terms_set = list(terms_set)
d_len = len(doc_list)
q_len = len(query_list)
w_len = len(terms_set)
p_q_d = np.ones([q_len, d_len], dtype=np.float64)
K = 32
iteration = 35

In [7]:
# initialization
p_k_w_d = np.zeros([K, w_len, d_len], dtype=float)
p_w_k = np.zeros([w_len, K], dtype=float)
p_k_d = np.zeros([K, d_len], dtype=float)
w_d = np.zeros([w_len, d_len], dtype=float)
# P(wi | Tk) random
for k in tqdm(range(K)):
    sum = 0
    for w in range(w_len):
        temp = random.uniform(1.0, 10.0)    
        p_w_k[w][k] = temp
        sum += temp
    for w in range(w_len):
        p_w_k[w][k] /= sum
# P(Tk | dj) uniform
p_k_d.fill(1/K)

100%|██████████| 32/32 [00:00<00:00, 332.25it/s]


In [8]:
# w_d
for d in tqdm(range(d_len)):
    d_counter = Counter(doc[d])
    for w in range(w_len):
        w_d[w][d] = d_counter[terms_set[w]]

100%|██████████| 14955/14955 [00:19<00:00, 755.39it/s]


In [9]:
# BackGround
bg = np.zeros(w_len, dtype=int)
def BG():
    for w in tqdm(range(w_len)):
        bg[w] = np.nansum(w_d[w])

In [10]:
BG()

100%|██████████| 2619/2619 [00:00<00:00, 25313.00it/s]


In [11]:
# Loss
@jit(nopython= True)
def loss(pwk, pkd, wd, wl, dl):
    pk = np.dot(pwk, pkd) # w * d
    pk = pk / dl + 0.1
    pk = np.log(pk)
    loss = 0.0
    for w in range(wl):
        for d in range(dl):
            loss += wd[w][d] * pk[w][d]
    return loss

In [12]:
@jit(nopython= True)
def E_step(pkwd, pwk, pkd, wl, dl, k):
    e_den = np.dot(pwk, pkd) # w * d
    # E_Step
    for w in range(wl):
        for d in range(dl):
            for c in range(k):
                if(not(e_den[w][d] == 0)):
                    pkwd[c][w][d] = (pwk[w][c] * pkd[c][d] / e_den[w][d])
    return pkwd

In [13]:
@jit(nopython= True)
def M_step(pwk, wd, pkwd, k, wl, dl):
    # M_Step
    for c in range(k):
        m_temp = np.copy(wd)
        for w in range(wl):
            for d in range(dl):
                m_temp[w][d] *= pkwd[c][w][d]
        m_den = np.sum(m_temp)
        if(not(m_den == 0)):
            for w in range(wl):
                    pwk[w][c] = np.sum(m_temp[w]) / m_den
    return pwk

In [14]:
@jit(nopython= True)
def P_K_D(pkd, wd, pkwd, k, wl, dl, dl_list):
    for c in range(k):
        for d in range(dl):
            sum = 0.0
            for w in range(wl):
                sum += wd[w][d] * pkwd[c][w][d]
            pkd[c][d] = sum / dl_list[d]
    return pkd

In [15]:
# PLSA Iteration
full_time = time.time()
L_temp = 0
for i in range(iteration):
    print('Iteration {}:\n'.format(i+1))
    print('\nE_step_{}: '.format(i+1))
    e_start = time.time()
    p_k_w_d = E_step(p_k_w_d, p_w_k, p_k_d, w_len, d_len, K)
    print("Time = {} sec\n".format(time.time() - e_start))
    print('\nM_step_{}: '.format(i+1))
    m_start = time.time()
    p_w_k = M_step(p_w_k, w_d, p_k_w_d, K, w_len, d_len)
    print(" Time = {} sec\n".format(time.time() - m_start))
    print('P(Tk | dj)_{}: '.format(i+1))
    p_start = time.time()
    p_k_d = P_K_D(p_k_d, w_d, p_k_w_d, K, w_len, d_len, doc_len_list)
    print("Time = {} sec\n".format(time.time() - p_start))
    L = loss(p_w_k, p_k_d, w_d, w_len, d_len)
    print('Loss_{} = {}\n\n'.format(i+1, abs(L-L_temp)))
    L_temp = L
print('Full Extime = {} min\n'.format((time.time() - full_time) / 60))

Iteration 1:


E_step_1: 
Time = 13.649882078170776 sec


M_step_1: 
 Time = 12.073198556900024 sec

P(Tk | dj)_1: 
Time = 22.361243963241577 sec

Loss_1 = 12889384.59412215


Iteration 2:


E_step_2: 
Time = 4.931212425231934 sec


M_step_2: 
 Time = 8.505075931549072 sec

P(Tk | dj)_2: 
Time = 21.567670583724976 sec

Loss_2 = 0.015555208548903465


Iteration 3:


E_step_3: 
Time = 2.1868863105773926 sec


M_step_3: 
 Time = 8.468501806259155 sec

P(Tk | dj)_3: 
Time = 21.816114902496338 sec

Loss_3 = 0.031223611906170845


Iteration 4:


E_step_4: 
Time = 2.4079296588897705 sec


M_step_4: 
 Time = 8.577717542648315 sec

P(Tk | dj)_4: 
Time = 21.736992359161377 sec

Loss_4 = 0.07838798128068447


Iteration 5:


E_step_5: 
Time = 2.660743474960327 sec


M_step_5: 
 Time = 8.3782799243927 sec

P(Tk | dj)_5: 
Time = 21.961570978164673 sec

Loss_5 = 0.24729678593575954


Iteration 6:


E_step_6: 
Time = 2.4618325233459473 sec


M_step_6: 
 Time = 8.447790145874023 sec

P(Tk | dj)_6: 
Tim

KeyboardInterrupt: 

In [18]:
# query likelihood
alpha = 0.75
beta = 0.15
term_2_1 = np.matmul(p_w_k, p_k_d, dtype=float)
for q in tqdm(range(q_len)):
        for d in range(d_len):
            for w in query[q]:
                term_1 = alpha * w_d[terms_set.index(w)][d] / doc_len_list[d]
                term_2 = beta * term_2_1[terms_set.index(w)][d]
                term_3 = (1 - alpha - beta) * bg[terms_set.index(w)] / all_d_len
                p_q_d[q][d] *= (term_1 + term_2 + term_3)
# file output
fp = open("plsa_alpha_{}_beta_{}.txt".format(alpha, beta), "w")
fp.write("Query,RetrievedDocuments\n")
for q in range(q_len):
    fp.write('{},'.format(query_list[q]))
    for s in np.argsort(-p_q_d[q])[:1000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()

100%|██████████| 100/100 [03:27<00:00,  2.08s/it]
